# Home Assignment No. 3 ( Practice)

# fMRI Deep Learning analysis

* You are **HIGHLY RECOMMENDED** to read relevant documentation, e.g. for [python](https://docs.python.org/3/), [numpy](https://docs.scipy.org/doc/numpy/reference/), [matlpotlib](https://matplotlib.org/), [sklearn](https://scikit-learn.org/stable/) and [pytorch](https://pytorch.org/). Also remember that seminars, lecture slides, [Google](http://google.com) and [StackOverflow](https://stackoverflow.com/) are your close friends during this course (and, probably, whole life?).

* If you want an easy life, you have to use **BUILT-IN METHODS** of `sklearn` and `pytorch` libraries, as well as ready **NN BUILDING BLOCKS** from `pytorch.nn` module, instead of writing tons of your own code. There exists a class/method for almost everything you can imagine (related to this homework).

* To do this part of homework, you have to write **CODE** directly inside specified places inside notebook **CELLS**.

* In some problems you are asked to provide short discussion of the results. In these cases you have to create **MARKDOWN** cell with your comments right after the corresponding code cell.

* For every separate problem you can get only 0 points or maximal points for this problem. There are **NO INTERMEDIATE SCORES**. So make sure that you did everything required in the task

* Your **SOLUTION** notebook **MUST BE REPRODUCIBLE**, i.e. if the reviewer decides to execute all, after all the computation he will obtain exactly the same solution (with all the corresponding plots) as in your uploaded notebook. For this purpose, we suggest to fix random `seed` or (better) define `random_state=` inside every algorithm that uses some pseudorandomness.

* Your code must be clear to the reviewer. For this purpose, try to include neccessary comments inside the code. But remember: **GOOD CODE MUST BE SELF-EXPLANATORY** without any additional comments.

* Many `sklearn` algorithms support multithreading (Ensemble Methods, Cross-Validation, etc.). Check if the particular algorithm has `n_jobs` parameters and set it to `-1` to use all the cores.

To begin with, let's import the essential (for this assignment) libraries.

In [1]:
import os
import time
from tqdm import tqdm
import nibabel as nib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms

### Lets build  the full-size fMRI classification with autoencoder-based model (10 points)

#### To get maximum points you are to:
 1. train an autoencoder model **(2 pts)** and attach the trained model with model.state_dict **(1 pts)**.

 2. provide code for encoding fMRI sequences into sequences of 1D latent vectors **(1 pts)**.

 3. train three (separate) RNN models on the encoded fMRI sequences to detect SCHZ, BIPOLAR, and ADHD pathologies from CONTROL group, (**3 pts**, **1 pts** for each)
 
 4. obtain on cross-validation ROC AUC scores not less than:
    - **0.75** for SCHZ/CONTROL (**1 pts**)
    - **0.68** for BIPOLAR/CONTROL (**1 pts**)
    - **0.70** for ADHD/CONTROL (**1 pts**)
 

In [4]:
use_cuda = torch.cuda.is_available()

print("Torch version:", torch.__version__)
if use_cuda:
    print("Using GPU")
else:
    print("Not using GPU")

device = 0

Torch version: 1.4.0
Not using GPU


#### Part 1: train a convolutional **autoencoder model** on brain images from fMRI time series. 

1. Load training and validation data.
2. Implement an autoencoder model.
3. Train the autoencoder to a satisfactory reconstruction quality.
4. Save the model.

Notes:

* The best is to train autoencoder on the brain images sampled from the whole fMRI sequence at random time steps. However, for the sake of more fast and memory-efficient training, here you should use prepared tensor datasets:
    * **`tensors_train`** - tensor of brain images taken at the **0**th time step of each fMRI series - for **training**,
    * **`tensors_val`** - tensor of brain images taken at the **16**th time step of each fMRI series - for **validation**.

* Changes in functional activity in fMRI images may account only for a small fraction of the total signal. Therefore, it may be important to achieve a sufficiently high reconstruction quality to ensure that they are properly reflected in a latent representation.

* Try to avoid too complex and wide autoencoder architecture - it would be very likely overkill.

In [3]:
# Load tensors with training and validation data

tensors_train = torch.load("/content/drive/My Drive/NeuroML/tensors_train")
tensors_val = torch.load("/content/drive/My Drive/NeuroML/tensors_val")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/NeuroML/tensors_train'

In [ ]:
# implement Autoencoder model

class Autoencoder3d(nn.Module):
    def __init__(self):
        super(self.__class__, self).__init__()

        ### YOUR CODE HERE
        
        self.encoder = ### YOUR CODE HERE
        self.decoder = ### YOUR CODE HERE
        
    def encode(self, x):
        z = self.encoder(x) 
        return z
    
    def decode(self, z):
        x_rec = self.decoder(z)
        return x_rec

    def forward(self, x):        
        z = self.encode(x)
        x_rec = self.decode(z)
        return x_rec, z

In [ ]:
# implement all needed train functions & train autoencoder

### YOUR CODE HERE

In [ ]:
# save trained autoencoder model

name = "fMRI_Autoencoder"
print(name)
torch.save({
    'epoch': 200,
    'model_state_dict': AE.state_dict(),
    'optimizer_state_dict': AE_opt.state_dict(),
}, "/content/drive/My Drive/NeuroML/{}.pth".format(name))

fMRI_Autoencoder


#### Part 2: encode the full-size fMRI sequence into a sequence of one-dimensional latent vectors with the pretrained encoder. 

1. Load the training data and prepare training dataset.
2. Iterate through all fMRI sequences and transform each image in a sequence into a latent vector.
3. Construct a tensor dataset with encoded sequences and corresponding labels.

Notes:
<!-- * Whole fMRI sequences dataset is located at `folder_path = '/content/drive/My Drive/NeuroML/func/'` -->
* Brain images in training dataset for autoencoder were individually normalized to **(0, 1)** range. Use **`RescaleIntensity`** transform to achieve the same effect when encoding the data.
* You do not have to use whole fMRI sequences. Probably you will find out that **64**, **128**, or **160** time steps are enough to capture important temporal patterns.


In [ ]:
# transforms
import warnings

class ToTensor(object):
    def __call__(self, img):
        return torch.FloatTensor(img)

class RescaleIntensity(object):
    """Rescale intensity values to a certain range.

    Args:
        out_min_max: Range :math:`(n_{min}, n_{max})` of output intensities.
        percentiles: Percentile values of the input image that will be mapped
            to :math:`(n_{min}, n_{max})`. They can be used for contrast
            stretching, as in `this scikit-image example`_. For example,
            Isensee et al. use ``(0.05, 99.5)`` in their `nn-UNet paper`_.
        masking_method: See
            :py:class:`~torchio.transforms.preprocessing.normalization_transform.NormalizationTransform`.
        p: Probability that this transform will be applied.
    """
    def __init__(self, out_min_max, percentiles=(0, 100), masking_method=None):
#             masking_method: TypeMaskingMethod=None,
        super().__init__()
        self.out_min, self.out_max = out_min_max
        self.percentiles = percentiles
        self.masking_method = masking_method
        
    def __call__(self, sample):
        if self.masking_method is None:
            mask = torch.ones_like(sample)
        else:
            mask = self.masking_method(sample)
        return self.rescale(sample, mask)

    def rescale(self, tensor, mask):
        array = tensor.numpy()
        mask = mask.numpy()
        values = array[mask]
        cutoff = np.percentile(values, self.percentiles)
        np.clip(array, *cutoff, out=array)
        array -= array.min()  # [0, max]
        array_max = array.max()
        if array_max == 0:
            message = (f'Rescaling image not possible due to division by zero')
            warnings.warn(message)
            return tensor
        array /= array.max()  # [0, 1]
        out_range = self.out_max - self.out_min
        array *= out_range  # [0, out_range]
        array += self.out_min  # [out_min, out_max]
        return torch.from_numpy(array)

In [ ]:
# implement fMRI Dataset/Dataloader class

### YOUR CODE HERE

In [ ]:
# load & transform whole fMRI sequences data

transform = transforms.Compose([
    ToTensor(),
    RescaleIntensity((0, 1), masking_method=lambda x: x > 0),
])

### YOUR CODE HERE

100%|██████████| 263/263 [00:00<00:00, 336516.76it/s]


262 image files found.


In [ ]:
# encode fMRI sequences into the one-dimensional vector time series with pretrained autoencoder

Z, Y = [], []
with torch.no_grad():
    for i in tqdm(range(len(dataset))):
        x, y = dataset[i]

        ### YOUR CODE HERE
        z = z.cpu().detach().numpy().reshape(len(x), -1)
        
        Z.append(z)
        Y.append(y) 
Z = np.stack(Z, axis=0).transpose(0, 2, 1)
Y = np.array(Y)

Z.shape, Y.shape

100%|██████████| 262/262 [15:45<00:00,  3.61s/it]


((262, 6144, 128), (262,))

#### Part 3: train a RNN models on the sequence of one-dimensional latent vectors to predict each of three pathologies (SCHZ (Schizophrenia), BIPOLAR, ADHD). 

1. Implement a RNN model.
2. Train RNN model and measure ROC AUC score on 3-fold cross-validation for 3 pathology vs control classification tasks. 

Notes:
* Overfitting is highly probable, so you may want to experiment with various regularization strategies (dropout, weight decay).


In [ ]:
# implement RNN model

class RNNModel(nn.Module):
    def __init__(self,):
        super(self.__class__, self).__init__()
        
        ### YOUR CODE HERE
        
    def forward(self, x):
      
        ### YOUR CODE HERE

        return x

In [ ]:
# inplement all needed training functions

### YOUR CODE HERE

In [ ]:
# use 3-fold cross-validation with random_state 42 to estimate model performance

cv = StratifiedKFold(n_splits=3, random_state=42)

In [ ]:
problem = "SCHZ"
idx = np.argwhere((Y == problem) + (Y == "CONTROL")).ravel()
tensor_latentvecs = data.dataset.TensorDataset(
    torch.FloatTensor(Z[idx]),
    torch.LongTensor(Y[idx] == problem)
)
# train on latentvectors dataset

### YOUR CODE HERE

In [ ]:
problem = "BIPOLAR"
idx = np.argwhere((Y == problem) + (Y == "CONTROL")).ravel()
tensor_latentvecs = data.dataset.TensorDataset(
    torch.FloatTensor(Z[idx]),
    torch.LongTensor(Y[idx] == problem)
)
# train on latentvectors dataset

### YOUR CODE HERE

In [ ]:
problem = "ADHD"
idx = np.argwhere((Y == problem) + (Y == "CONTROL")).ravel()
tensor_latentvecs = data.dataset.TensorDataset(
    torch.FloatTensor(Z[idx]),
    torch.LongTensor(Y[idx] == problem)
)
# train on latentvectors dataset

### YOUR CODE HERE